In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Crowdshipping Pricing Model

This module implements an optimization model for crowdshipping platforms that:
- Match drivers with customer orders  
- Build driver routes
- Determine optimal pricing strategies

The model uses mixed-integer programming (MIP) with OR-Tools to maximize
platform profit while satisfying constraints for time windows, capacity,
willingness-to-pay (WTP), and expected compensation (ETP).
"""

# ============================================================================
# PACKAGE INSTALLATION
# ============================================================================

!pip install matplotlib
!pip install networkx
!pip install numpy
!pip install haversine
!pip install cartopy
!pip install geopandas
!pip install datetime
!pip install openpyxl
!pip install ortools 
!pip install fiona shapely pyproj rtree
!pip install --upgrade fiona geopandas

# ============================================================================
# IMPORTS
# ============================================================================

import os
import numpy as np
import networkx as nx
import ortools
from ortools.linear_solver import pywraplp
from haversine import haversine
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import geopandas as gpd
import fiona
import pandas as pd
import matplotlib.cm as cm
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
from datetime import datetime
from datetime import timedelta
from openpyxl import load_workbook

# ============================================================================
# REPOSITORY SETUP
# ============================================================================

# Copy the repository
repo_url = "https://github.com/rimchmielowitz/Pricing.git"
repo_name = "Pricing"

if not os.path.exists(repo_name):
    !git clone --recurse-submodules {repo_url}
    print("✅ Repository wurde geklont.")
else:
    print("✅ Repository existiert bereits.")

# Change to repository folder
os.chdir(repo_name)

# SHAPE_RESTORE_SHX aktivieren
os.environ["SHAPE_RESTORE_SHX"] = "YES"

# ============================================================================
# CONFIGURATION AND DATA PATHS
# ============================================================================

# Model parameters
M = 10000  # Big-M parameter for linearization constraints
time_format_output = '%H:%M'  # Time display format

# Repository configuration
repo_url = "https://github.com/rimchmielowitz/Pricing.git"
repo_name = "Pricing"

# Data paths for Berlin test instance
data_folder = os.path.join(os.getcwd(), "data_Berlin")
data1 = os.path.join(data_folder, "TWD.csv")      # Main instance data (requests, drivers, parameters)
data2 = os.path.join(data_folder, "districts.csv")  # Berlin district information for visualization

# ============================================================================
# OPTIMIZATION MODEL
# ============================================================================

def pricing_model(n, m, A, A_, H, WTP, ETP, R, d2, V, d, t, s, l, a, b, M):
    """
    Main pricing optimization model for crowdshipping platform.
    
    MATHEMATICAL MODEL OVERVIEW:
    This implements a Vehicle Routing Problem with Pickup and Delivery (VRPPD)
    combined with dynamic pricing. The platform acts as intermediary between
    customers (who have delivery requests) and drivers (who fulfill them).
    
    DECISION VARIABLES:
    - x[i,j,h]: Binary - Driver h travels from node i to node j
    - S[i,h]: Integer - Start time of service at node i by driver h  
    - L[i,h]: Integer - Load of vehicle h at node i
    - z[i]: Binary - Request i is rejected (goes to request bank)
    - p[i,j,h]: Continuous - Price per km charged for arc (i,j) by driver h
    - c[i,j,h]: Continuous - Compensation per km paid to driver h for arc (i,j)
    
    OBJECTIVE: Maximize platform profit = Total Revenue - Total Costs
    
    Args:
        n (int): Number of customer requests
        m (int): Number of available drivers  
        A (list): Pickup nodes (nodes 1 to n)
        A_ (list): Delivery nodes (nodes n+1 to 2n)
        H (list): Set of drivers (1 to m)
        WTP (dict): Customer willingness-to-pay per request [€]
        ETP (dict): Driver expected compensation per km [€/km] 
        R (dict): Vehicle capacity per driver [units]
        d2 (dict): Direct pickup-to-delivery distance per request [km]
        V (list): All nodes in network
        d (dict): Distance matrix between all node pairs [km]
        t (dict): Travel time matrix [minutes]
        s (dict): Service times at nodes [minutes]
        l (dict): Load changes at nodes [units]
        a (dict): Time window earliest start [minutes from midnight]
        b (dict): Time window latest start [minutes from midnight]
        M (int): Big-M parameter for linearization constraints
    
    Returns:
        tuple: (objective_value, optimal_L, optimal_S, optimal_c, optimal_p, 
                optimal_x, optimal_z, runtime) or None if infeasible
    """
    
    # Initializing OR-Tools Solver (MIP-Optimization)
    solver = pywraplp.Solver.CreateSolver('SCIP')

    if not solver:
        print("Solver konnte nicht erstellt werden!")
        return None

    # ========================================================================
    # DECISION VARIABLES
    # ========================================================================
    
    # Binary variable: 1 if driver h travels from node i to j, 0 otherwise
    x = {}
    for (i, j) in E:
        for h in H:
            x[i, j, h] = solver.IntVar(0, 1, f'x[{i},{j},{h}]')

    # Service start time at node i for driver h
    S = {}
    for i in V:
        for h in H:
            S[i, h] = solver.IntVar(0, solver.infinity(), f'S[{i},{h}]')

    # Load of vehicle h at node i
    L = {}
    for i in V:
        for h in H:
            L[i, h] = solver.IntVar(0, solver.infinity(), f'L[{i},{h}]')

    # Binary variable: 1 if request i is not served, 0 otherwise
    z = {}
    for i in A:
        z[i] = solver.IntVar(0, 1, f'z[{i}]')

    # Price per unit distance for arc (i,j) and driver h
    p = {}
    for (i, j) in E:
        for h in H:
            p[i, j, h] = solver.NumVar(0, solver.infinity(), f'p[{i},{j},{h}]')

    # Compensation per unit distance for arc (i,j) and driver h
    c = {}
    for (i, j) in E:
        for h in H:
            c[i, j, h] = solver.NumVar(0, solver.infinity(), f'c[{i},{j},{h}]')

    # ========================================================================
    # OBJECTIVE FUNCTION
    # ========================================================================
    
    # Maximize the profit (price - costs)
    solver.Maximize(
        solver.Sum(p[i, j, h] * d2[i] - c[i, j, h] * d[i, j] 
                  for (i, j) in E for h in H if i != j)
    )

    # ========================================================================
    # CONSTRAINTS
    # ========================================================================
    
    # PRICING CONSTRAINTS
    # Constraint (02): Price cannot exceed customer's willingness-to-pay
    # Ensures customers only pay what they're willing to pay
    for h in H:
        for i in A:
            for j in pickup_delivery:
                if i != j:
                    solver.Add(p[i, j, h] <= WTP[i] / d2[i] + (1 - x[i, j, h]) * M)

    # COMPENSATION CONSTRAINTS  
    # Constraint (03): Driver compensation must meet minimum expectations
    # Ensures drivers get at least their expected compensation per km
    for h in H:
        for i in origin_pickup_delivery[h]:
            for j in pickup_delivery:
                if i != j:
                    solver.Add(c[i, j, h] >= ETP[h] * x[i, j, h])

    # REVENUE SHARING CONSTRAINTS
    # Constraint (04): Price must cover compensation (platform profit ≥ 0)
    # Ensures platform can pay drivers from customer payments
    for h in H:
        for i in A:
            for j in pickup_delivery:
                if i != j:
                    solver.Add(p[i, j, h] >= c[i, j, h])

    # REQUEST ASSIGNMENT CONSTRAINTS
    # Constraint (05): Each request is either served or rejected
    # Every customer request must be handled (either matched or put in request bank)
    for i in A:
        solver.Add(solver.Sum(x[i, j, h] for j in pickup_delivery for h in H if i != j) + z[i] == 1)

    # PICKUP-DELIVERY MATCHING CONSTRAINTS
    # Constraint (06): Same driver must handle pickup and delivery of same request
    # Flow conservation: if driver picks up request i, same driver must deliver it
    for h in H:
        for i in A:
            solver.Add(
                solver.Sum(x[i, j, h] for j in Vh[h] if (i, j, h) in x and i != j) - 
                solver.Sum(x[j, n+i, h] for j in Vh[h] if (j, n+i, h) in x and i != n+i) == 0
            )

    # DRIVER ROUTE CONSTRAINTS
    # Constraint (07): Each active driver must start from their origin
    # Ensures proper tour start point for each driver
    for h in H:
        solver.Add(solver.Sum(x[origin[h], j, h] for j in pickup_destination[h]) == 1)

    # Constraint (08): Each active driver must end at their destination  
    # Ensures proper tour end point for each driver
    for h in H:
        solver.Add(solver.Sum(x[i, tau_[h], h] for i in delivery_origin[h] if i != tau_[h]) == 1)

    # FLOW CONSERVATION CONSTRAINTS
    # Constraint (09): Flow conservation at pickup/delivery nodes
    # Driver arriving at node must also leave that node (except origins/destinations)
    for j in pickup_delivery:
        for h in H:
            solver.Add(solver.Sum(x[i, j, h] for i in V if i != j) - 
                      solver.Sum(x[j, i, h] for i in V if i != j) == 0)
    
    # TIME CONSISTENCY CONSTRAINTS
    # Constraint (10): Time sequencing along routes
    # If driver goes from i to j, arrival time at j ≥ departure time from i + travel time
    for (i, j) in E:
        for h in H:
            solver.Add(S[i, h] + s[i] + t[i, j] <= S[j, h] + (1 - x[i, j, h]) * M)

    # TIME WINDOW CONSTRAINTS
    # Constraint (11): Service must occur within customer/driver time windows
    # Each node has specific operating hours that must be respected
    for i in V:
        for h in H:
            solver.Add(S[i, h] >= a[i])  # Not before earliest time
            solver.Add(S[i, h] <= b[i])  # Not after latest time

    # PRECEDENCE CONSTRAINTS
    # Constraint (12): Pickup must occur before delivery for same request
    # Logical constraint: can't deliver before picking up
    for i in A:
        for h in H:
            solver.Add(S[i, h] <= S[n + i, h])

    # CAPACITY CONSTRAINTS
    # Constraint (13): Vehicle load tracking along route
    # Load after visiting node j = load at i + load change at j
    for (i, j) in E:
        for h in H:
            solver.Add(L[i, h] + l[j] <= L[j, h] + (1 - x[i, j, h]) * M)

    # Constraint (14): Vehicle capacity limits
    # Load cannot exceed vehicle capacity at any point
    for i in V:
        for h in H:
            solver.Add(L[i, h] <= R[h])

    # BOUNDARY CONDITIONS
    # Constraint (15): Vehicles start and end empty
    # Drivers begin and end tours with no load
    for h in H:
        solver.Add(L[origin[h], h] == 0)  # Start empty
        solver.Add(L[destination[h], h] == 0)  # End empty

    # ========================================================================
    # LINEARIZATION CONSTRAINTS
    # ========================================================================
    
    # LINEARIZATION FOR PRICING VARIABLES
    # These constraints ensure that pricing variables are only active when 
    # the corresponding route is chosen (x[i,j,h] = 1)
    
    # Constraint (23): If route not chosen (x=0), then price must be 0
    for (i, j) in E:
        for h in H:
            if i != j:
                solver.Add(p[i, j, h] <= M * x[i, j, h])
    
    # LINEARIZATION FOR COMPENSATION VARIABLES
    # Constraint (24): If route not chosen (x=0), then compensation must be 0
    for (i, j) in E:
        for h in H:            
            solver.Add(c[i, j, h] <= M * x[i, j, h])
   
    # ADDITIONAL LINEARIZATION CONSTRAINTS
    # Redundant constraints for solver stability and bounds
    for (i, j) in E:
        for h in H:
            if i != j:
                solver.Add(p[i, j, h] <= M * x[i, j, h])  # Price = 0 if route not used
                solver.Add(p[i, j, h] >= 0)  # Price cannot be negative

    for (i, j) in E:
        for h in H:
            if i != j:
                solver.Add(c[i, j, h] <= M * x[i, j, h])  # Compensation = 0 if route not used
                solver.Add(c[i, j, h] >= 0)  # Compensation cannot be negative
    
    # ========================================================================
    # ADDITIONAL CONSTRAINTS
    # ========================================================================
    
    # DESTINATION NODE CONSTRAINTS
    # Constraint (25): Each origin node must have exactly one outgoing arc
    # Ensures each driver's origin is properly connected
    for i in tau[1:]:
        solver.Add(solver.Sum(x[i, j, h] for h in H for j in V if i != j) == 1)
    
    # Constraint (26): No outgoing arcs from destination nodes
    # Prevents drivers from continuing after reaching their destination
    for h in H:
        solver.Add(solver.Sum(x[i, j, h] for i in tau_[1:] for h in H for j in V if i != j) == 0)

    # ========================================================================
    # SOLVE MODEL
    # ========================================================================
    
    status = solver.Solve()

    if status == pywraplp.Solver.OPTIMAL:
        optimal_x = {(i, j, h): x[i, j, h].solution_value() for (i, j) in E for h in H}
        optimal_S = {(i, h): S[i, h].solution_value() for i in V for h in H}
        optimal_L = {(i, h): L[i, h].solution_value() for i in V for h in H}
        optimal_z = {i: z[i].solution_value() for i in A}
        optimal_p = {(i, j, h): p[i, j, h].solution_value() for (i, j) in E for h in H}
        optimal_c = {(i, j, h): c[i, j, h].solution_value() for (i, j) in E for h in H}

        OF_p = solver.Objective().Value()
        Runtime1 = solver.WallTime()
        
        return OF_p, optimal_L, optimal_S, optimal_c, optimal_p, optimal_x, optimal_z, Runtime1
    else:
        return None


# ============================================================================
# DATA PROCESSING FUNCTIONS
# ============================================================================

def read_data_create_sets(data1):
    """
    Read and process data from CSV files to create optimization sets.
    
    DATA STRUCTURE EXPLANATION:
    - Requests: Each request i has pickup node i and delivery node i+n
    - Drivers: Each driver h has origin node (2n+h) and destination node (2n+m+h)
    - Time windows: Each node has [a[i], b[i]] availability window
    - Capacity: Each request has load l[i], each driver has capacity R[h]
    
    SETS CREATED:
    - A: Pickup nodes {1, 2, ..., n}
    - A_: Delivery nodes {n+1, n+2, ..., 2n}  
    - H: Drivers {1, 2, ..., m}
    - V: All nodes = A ∪ A_ ∪ origins ∪ destinations
    - E: All edges between nodes (except self-loops)
    
    Args:
        data1 (str): Path to TWD.csv file containing all instance data
    
    Returns:
        tuple: All processed data and sets needed for the optimization model
    """
    delimiter = ';'
    df = pd.read_csv(data1, delimiter=delimiter, header=0, 
                    parse_dates=['Time a', 'Time b'], date_format="%H:%M")
    dfdistricts = pd.read_csv(data2, delimiter=delimiter, header=0)
    
    # Extract basic parameters
    m = int(df.at[0, "m"])
    n = int(df.at[0, "n"])
    
    # Process WTP and ETP factors
    if type(df.at[0, "WTP_Faktor"]) == np.float64:
        WTP_Faktor = df.at[0, "WTP_Faktor"]
    if type(df.at[0, "WTP_Faktor"]) == str:
        WTP_Faktor = df.loc[0:0, "WTP_Faktor"].str.replace(',', '.').astype(float)[0]
    
    if type(df.at[0, "ETP_Faktor"]) == np.float64:
        ETP_Faktor = df.at[0, "ETP_Faktor"]
    if type(df.at[0, "ETP_Faktor"]) == str:
        ETP_Faktor = df.loc[0:0, "ETP_Faktor"].str.replace(',', '.').astype(float)[0]
    
    Speed = int(df.at[0, "Speed"])
    
    # Define row ranges
    start_rowpu = 0                                                        
    start_rowdel = start_rowpu + n                                          
    start_rowdriver = start_rowdel + n
    
    # Create node sets
    A = df.loc[start_rowpu:start_rowdel-1, 'i'].astype(int).tolist()
    A_ = df.loc[start_rowdel:start_rowdriver-1, 'i'].astype(int).tolist()
    
    # Process coordinates
    AllX = df.loc[start_rowpu:start_rowdriver+2*m-1, 'Long'].str.replace(',', '.').astype(float).tolist()
    AllX.insert(0, 0.0)
    AllY = df.loc[start_rowpu:start_rowdriver+2*m-1, 'Lat'].str.replace(',', '.').astype(float).tolist()
    AllY.insert(0, 0.0)
    
    # Process driver capacities
    R2 = df.loc[start_rowdriver:start_rowdriver+m-1, 'R'].astype(int).tolist()
    R2.insert(0, 0)
    
    # Process district information
    district_names = {idx: value for idx, value in enumerate(
        dfdistricts.loc[0:14, 'Name'].astype(str), start=0)}
    district_coordx = dfdistricts.loc[start_rowpu:14, 'Lat'].str.replace(',', '.').astype(float).tolist()
    district_coordy = dfdistricts.loc[start_rowpu:14, 'Long'].str.replace(',', '.').astype(float).tolist()
    
    # Create parameter dictionaries
    l = {idx: int(value) for idx, value in enumerate(
        df.loc[start_rowpu:start_rowdriver+2*m-1, 'l'].astype(int), start=1)}
    a = {idx: int(value) for idx, value in enumerate(
        df.loc[start_rowpu:start_rowdriver+2*m-1, 'a'].astype(int), start=1)}
    b = {idx: int(value) for idx, value in enumerate(
        df.loc[start_rowpu:start_rowdriver+2*m-1, 'b'].astype(int), start=1)}
    s = {idx: int(value) for idx, value in enumerate(
        df.loc[start_rowpu:start_rowdriver+2*m-1, 's'].astype(int), start=1)}
    
    # Create driver set
    H = list(range(1, m + 1))
    
    # Process time windows
    TW_LB = {idx: pd.to_datetime(value, format="%H:%M:%S").strftime(time_format_output) 
             for idx, value in enumerate(
                 df.loc[start_rowpu:start_rowdriver+2*m-1, 'Time a'], start=1)}

    TW_UB = {idx: pd.to_datetime(value, format="%H:%M:%S").strftime(time_format_output) 
             for idx, value in enumerate(
                 df.loc[start_rowpu:start_rowdriver+2*m-1, 'Time b'], start=1)}
    
    # Create capacity dictionary
    R = {h: R2[h] for h in H}
    
    # Create origin and destination node sets
    tau = [2*n + h for h in H]  # origin nodes of couriers
    tau.insert(0, 0)
    tau_ = [2*n + m + h for h in H]  # destination nodes of couriers
    tau_.insert(0, 0)
    
    # Calculate time window durations
    ZF = [b[i+m] - a[i] for i in tau[1:]]
    
    # Create comprehensive node sets
    V = A + A_ + tau[1:] + tau_[1:]     
    pickup_delivery = A + A_  # all pickup and delivery nodes
    
    # Set of destinations in dependence of driver h
    destination = {h: tau_[h] for h in H}
    # Set of origins in dependence of driver h
    origin = {h: tau[h] for h in H}
    # Unified set of pickups and destinations in dependence of driver h
    pickup_destination = {h: A + [destination[h]] for h in H}
    # Unified set of origins and drop-offs in dependence of driver h
    delivery_origin = {h: [origin[h]] + A_ for h in H}
    delivery_destination = {h: A_ + [destination[h]] for h in H}
    # Set of all nodes in dependence of driver h
    Vh = {h: [origin[h]] + A + A_ + [destination[h]] for h in H}
    origin_pickup_delivery = {h: [origin[h]] + A + A_ for h in H}
    pickup_delivery_destination = {h: pickup_delivery + [destination[h]] for h in H}
    origin_delivery_destination = A_ + tau[1:] + tau_[1:]
    
    # All edges except edges from one node to the same node back
    E = [(i, j) for i in V for j in V if i != j]
    E1 = {h: [(i, j) for i in Vh[h] for j in Vh[h] if i != j] for h in H}
    
    # travel distance for link i-->j
    d = {(i, j): haversine((AllX[i], AllY[i]), (AllX[j], AllY[j])) for i, j in E}
    d2 = {i: haversine((AllX[i], AllY[i]), (AllX[i+n], AllY[i+n])) for i in A}
    d21 = {i: 0 for i in origin_delivery_destination}
    d2.update(d21)
    
    # Travel times
    t = {(i, j): Speed * d[i, j] for i, j in E}
    
    # Generate random ETP values
    np.random.seed(10)
    ETPa = 0.5                                                 
    ETPb = 1
    ETP = {h: round(np.random.uniform(ETPa, ETPb) * ETP_Faktor, 2) for h in H}
    
    # Generate random WTP values
    np.random.seed(10)
    WTPa = 5
    WTPb = 10
    WTP = {i: round(np.random.uniform(WTPa, WTPb) * WTP_Faktor, 2) for i in A}
    
    return (WTP_Faktor, ETP_Faktor, H, R, tau, tau_, V, pickup_delivery, ETP, WTP, ZF, 
            TW_LB, TW_UB, destination, origin, pickup_destination, delivery_origin, 
            delivery_destination, Vh, origin_pickup_delivery, pickup_delivery_destination, 
            E, E1, d, d2, t, m, n, A, A_, a, b, l, s, R2, AllX, AllY, Speed, 
            district_names, district_coordx, district_coordy)


# ============================================================================
# RESULTS PROCESSING
# ============================================================================

def results():
    """
    Extract and process optimization results for analysis.
    
    This function processes the optimal solution to extract meaningful
    business metrics and operational insights.
    
    EXTRACTED METRICS:
    - Route assignments and active drivers
    - Financial metrics (revenue, costs, margins)  
    - Operational metrics (matches, distances, efficiency)
    - Pricing decisions per request and driver compensation
    
    Returns:
        tuple: Comprehensive results including all key performance indicators
    """
    # ========================================================================
    # EXTRACT ACTIVE SOLUTION COMPONENTS
    # ========================================================================
    
    # Active routes: which driver travels which segments
    active_arcs = [(i, j, h) for i, j in E for h in H if optimal_x[i, j, h] > 0.99]
    active_prices = [(i, j, h) for i, j in E for h in H if optimal_p[i, j, h] > 0.99]
    active_costs = [(i, j, h) for i, j in E for h in H if optimal_c[i, j, h] > 0.099]   
    active_load = [(i, h) for i in V for h in H if optimal_L[i, h] > 0.99]
    
    # Rejected requests (sent to request bank)
    active_z = [i for i in A if optimal_z[i] > 0.99]
    active_Start = [(i, h) for i in V for h in H if optimal_S[i, h] > 0.99]
    
    # ========================================================================
    # CALCULATE OPERATIONAL METRICS
    # ========================================================================
    
    # Total distance traveled by all drivers
    td = sum(optimal_x[i, j, h] * d[i, j] for h in H 
             for i in origin_pickup_delivery[h] for j in pickup_delivery if i != j)
    
    # Driver utilization metrics
    Fahrerstopps = {h: sum(optimal_x[i, j, h] for i, j in E) for h in H}
    Multistop_touren = {h: Fahrerstopps[h] for h in H}
    Touren = {h: [(i, j) for i, j in E if optimal_x[i, j, h] > 0.99] for h in H}
    
    # Calculate number of requests per driver
    for h in H:
        if Fahrerstopps[h] >= 2:
            Multistop_touren[h] = (Fahrerstopps[h] - 1) / 2
        else:
            del Multistop_touren[h]
            del Touren[h]
    
    # Identify active drivers (those with assigned routes)
    active_driver = [h for h in Touren if Touren[h] != []]
    
    # Matching efficiency metrics
    possible_matches = len(A)  # Total number of requests
    matches = len(A) - sum(optimal_z[i] for i in A)  # Successfully matched requests
    used_driver = sum(optimal_x[i, j, h] for i in A_ for j in tau_[1:] for h in H)
    
    # ========================================================================
    # PROCESS ROUTE INFORMATION
    # ========================================================================
    
    # Convert 3D arcs to 2D for easier processing
    active_arcs2D = list()
    for a in active_arcs:
        active_arcs2D.extend([a[:2]])
    
    # Filter out origin-to-destination direct connections
    active = list()
    for (i, j) in active_arcs2D:
        if i in tau[1:] and j in tau_[1:]:
            pass  # Skip direct origin-to-destination moves
        else:
            active.append((i, j))
    
    # ========================================================================
    # EXTRACT FINANCIAL DECISIONS
    # ========================================================================
    
    # Extract active compensation decisions
    chosen_c = {(i, j, h): optimal_c[i, j, h] for i, j in E for h in H if optimal_c[i, j, h] > 0.099}
    chosen_cost = {h: chosen_c[i, j, h] for i, j, h in chosen_c}
    
    # Extract active pricing decisions  
    chosen_p = {i: round(optimal_p[i, j, h], 2) for i, j in E for h in H if optimal_p[i, j, h] > 0.099}
    opt_request_p = {i: round(chosen_p[i] * d2[i], 2) for i in chosen_p}  # Total price per request
    opt_driver_c = {h: round(chosen_c[i, j, h], 2) for i, j, h in chosen_c}  # Cost per km for driver
    
    # ========================================================================
    # CALCULATE FINANCIAL PERFORMANCE
    # ========================================================================
    
    # Platform revenue and costs
    Einnahmen = round(sum(optimal_p[i, j, h] * d2[i] for i, j in E for h in H), 2)
    Einnahmen_je_Tour = {h: round(sum(optimal_p[i, j, h] * d2[i] for i, j in E), 2) for h in H}
    Ausgaben = round(sum(optimal_c[i, j, h] * d[i, j] for i, j in E for h in H), 2)
    Ausgaben_je_Tour = {h: round(sum(optimal_c[i, j, h] * d[i, j] for i, j in E), 2) for h in H}
    
    # Profit margins per tour
    Marge_je_Tour = {h: Einnahmen_je_Tour[h] - Ausgaben_je_Tour[h] for h in active_driver}
    
    # Request-level financial analysis
    aktive_Auftraege = [i for i in chosen_p]
    Einnahmen_je_Auftrag = {i: d2[i] * chosen_p[i] for i in aktive_Auftraege}
    
    # Calculate driver distance per request (pickup + delivery distance)
    Fahrerdistanz_je_Auftrag = {
        i: d[i, j] + d[a, i] for i in aktive_Auftraege 
        for j in pickup_delivery for h in H for a in origin_pickup_delivery[h] 
        if (i, j) in active and (a, i) in active
    }
    
    # Calculate costs per request
    Ausgaben_je_Auftrag = {
        i: Fahrerdistanz_je_Auftrag[i] * chosen_c[i, j, h] 
        for i, j, h in chosen_c if i in aktive_Auftraege
    }
    
    # Calculate profit margin per request
    Marge_je_Auftrag = {
        i: (Einnahmen_je_Auftrag[i] - Ausgaben_je_Auftrag[i]) 
        for i in chosen_p
    }
    
    # Edge-level cost information
    Kantenkosten = {
        (i, j): chosen_c[i, j, h] for h in active_driver 
        for (i, j) in Touren[h] if j not in tau_[1:]
    }
    
    # ========================================================================
    # PRINT SOLUTION SUMMARY
    # ========================================================================
    
    print("=" * 60)
    print("OPTIMIZATION RESULTS SUMMARY")
    print("=" * 60)
    print(f"Total distance from origin to dropoff = {td:.2f} km")
    print(f"Active arcs (routes used): {len(active_arcs)} routes")
    print(f"Rejected requests: {len(active_z)} out of {possible_matches}")
    print(f"Successful matches: {matches} out of {possible_matches} ({matches/possible_matches*100:.1f}%)")
    print(f"Active drivers: {len(active_driver)} out of {len(H)} ({len(active_driver)/len(H)*100:.1f}%)")
    print(f"Platform profit (ZFW): €{round(OF_p, 2)}")
    print()
    
    print("DRIVER UTILIZATION:")
    print(f"Requests per active driver: {Multistop_touren}")
    print(f"Active driver IDs: {active_driver}")
    print()
    
    print("FINANCIAL BREAKDOWN:")
    print(f"Total revenue: €{Einnahmen}")
    print(f"Total costs: €{Ausgaben}")
    print(f"Total margin: €{Einnahmen - Ausgaben}")
    print(f"Margin per request: {Marge_je_Auftrag}")
    print()
    
    print("PRICING DECISIONS:")
    for i in chosen_p:
        utilization_rate = (chosen_p[i] / (WTP[i] / d2[i])) * 100
        print(f"Request {i}: WTP=€{WTP[i]:.2f}, Charged=€{opt_request_p[i]:.2f}, "
              f"Rate=€{chosen_p[i]:.2f}/km (WTP utilization: {utilization_rate:.1f}%)")
    
    print()
    print("DRIVER COMPENSATION:")
    for h in active_driver:
        compensation_ratio = (opt_driver_c[h] / ETP[h]) if ETP[h] > 0 else 0
        print(f"Driver {h}: ETP=€{ETP[h]:.2f}/km, Paid=€{opt_driver_c[h]:.2f}/km "
              f"(Premium: {compensation_ratio:.1f}x ETP)")
    
    print("=" * 60)
    
    return (active_arcs, active_prices, active_costs, active_load, active_z, active_Start, 
            Touren, active_driver, active_arcs2D, active, chosen_c, chosen_cost, chosen_p, 
            opt_request_p, opt_driver_c, Einnahmen, Ausgaben, used_driver, possible_matches, 
            matches, Kantenkosten, Marge_je_Auftrag, Marge_je_Tour, aktive_Auftraege)


def variables_part2():
    """
    Process additional variables for visualization and detailed analysis.
    
    TOUR PROCESSING:
    This function takes the raw optimization results and processes them into
    meaningful tour sequences for drivers. It uses NetworkX to find the 
    longest path through each driver's assigned nodes, ensuring proper
    sequencing of pickup and delivery operations.
    
    EFFICIENCY METRICS:
    Calculates various efficiency measures including:
    - Detour ratios (extra distance vs. direct routes)
    - Driver surplus (compensation above minimum expected)
    - Customer surplus (savings vs. willingness-to-pay)  
    - Environmental impact (total km driven vs. optimal)
    
    Returns:
        tuple: Processed tour sequences, timing information, and efficiency metrics
    """
    # ========================================================================
    # PROCESS TOUR SEQUENCES
    # ========================================================================
    
    # Extract all nodes that are part of active routes
    active_nodes = []
    for a in active:
        active_nodes.extend(a[:1])
        active_nodes.extend(a[1:])
    active_nodes = list(dict.fromkeys(active_nodes))
    
    # Use NetworkX to find proper tour sequences
    # This ensures pickup-delivery precedence and creates logical tour order
    Touren_neu = {h: tuple for h in Touren}
    for h in Touren:
        edgars = [(i, j) for (i, j) in Touren[h]]
        GTouren = nx.DiGraph()
        GTouren.add_edges_from(edgars)
        longest_path = nx.dag_longest_path(GTouren)  # Find proper sequence
        graph_after = list(zip(longest_path[:-1], longest_path[1:]))
        Touren_neu[h] = graph_after
    
    # Create ordered node sequences for each driver tour
    Touren_sorted = {h: list() for h in Touren_neu}
    for h in Touren_neu:
        liste = []
        for a in Touren_neu[h]:
            list(a)
            liste.append(a[:1])
        liste.append(a[1:])    
        liste = [i[0] for i in liste]
        Touren_sorted[h] = liste
    
    # ========================================================================
    # CALCULATE TIMING INFORMATION
    # ========================================================================
    
    # Extract optimal start times for visualization
    opt_S_driver = {(i, h): optimal_S[i, h] for h in Touren_sorted for i in Touren_sorted[h]}
    Starting_times = {i: int(optimal_S[i, h]) for h in Touren_sorted for i in Touren_sorted[h]}
    
    # Convert start times to readable format (HH:MM)
    Starting_times_formatted = {}
    for key, value in Starting_times.items():
        time_obj = timedelta(minutes=value)
        time_str = (datetime.min + time_obj).strftime('%H:%M')
        Starting_times_formatted[key] = time_str
    
    # ========================================================================
    # CREATE DRIVER ACTIVITY STATUS
    # ========================================================================
    
    # Track which drivers are active vs. idle
    # Negative values indicate idle drivers
    activitis = list()
    for h in H:
        if h not in active_driver:
            not_driving = h * -1  # Mark idle drivers with negative ID
            activitis.append(not_driving)
        else:
            activitis.append(h)
    activitis.insert(0, 0)
    
    # ========================================================================
    # PROCESS DRIVER TIME WINDOWS
    # ========================================================================
    
    # Extract time windows for active drivers only
    used_origins = {h: origin[h] for h in origin if h in active_driver}
    TW_LB_driver = {h: str for h in used_origins}
    for h, i in used_origins.items():
        TW_LB_driver[h] = TW_LB[i]  # Earliest start time for driver h
    
    used_destinations = {h: destination[h] for h in destination if h in active_driver}
    TW_UB_driver = {h: str for h in used_destinations}
    for h, i in used_destinations.items():
        TW_UB_driver[h] = TW_UB[i]  # Latest end time for driver h
    
    # ========================================================================
    # CALCULATE EFFICIENCY AND SURPLUS METRICS
    # ========================================================================
    
    # Extract travel times for active routes
    active_t = {i: t[i, j] for i, j in active}
    
    # Filter WTP and ETP for active participants only
    used_ETP = {h: ETP[h] for h in active_driver}
    used_WTP = {i: WTP[i] for i in active_nodes if i in A}
    
    # Create mapping: which driver serves which request
    Auftrag_Fahrer = {node: driver for driver in Touren_sorted for node in Touren_sorted[driver]}
    
    # CUSTOMER SURPLUS: How much customers save vs. their WTP
    Senders_surplus = round(sum(WTP[i] for i in aktive_Auftraege) - Einnahmen, 2)
    
    # DRIVER SURPLUS: How much drivers earn above their minimum ETP
    Couriers_surplus = round(Ausgaben - sum(ETP[h] * d[i, j] * optimal_x[i, j, h] 
                                          for i, j in E for h in H 
                                          if i in origin_pickup_delivery[h] and j in pickup_delivery), 2)
    
    # ========================================================================
    # CALCULATE DETOUR AND EFFICIENCY METRICS
    # ========================================================================
    
    # Calculate detours: extra distance vs. direct origin-to-destination
    Umweg = {h: d[Touren_sorted[h][-2], destination[h]] for h in Touren_sorted}
    Umwege_summe = round(sum(Umweg[h] for h in Umweg), 2)  # Total detour distance
    
    # Total distance driven per tour
    ges_km_je_Tour = {h: round(sum(d[i] for i in Touren_neu[h]), 2) for h in Touren_neu}
    
    # Direct distance from each driver's origin to destination
    origin_dest_km = {h: round(d[origin[h], destination[h]], 2) for h in H}
    
    # Calculate detour ratios
    if sum(ges_km_je_Tour[h] for h in Touren_sorted) > 0:
        Umweg_ges_km = round(Umwege_summe / sum(ges_km_je_Tour[h] for h in Touren_sorted), 2)
    else:
        Umweg_ges_km = 0
    
    # Detour ratio per individual tour
    Umweg_Tour_Verhältnis = {h: round(Umweg[h] / ges_km_je_Tour[h], 2) for h in Touren_sorted}
    
    # Direct pickup-to-delivery distance per request
    km_Auftrag = {i: d[i, i+n] for i in A}
    
    # Relative detour per tour (vs. direct origin-destination)
    Umweg_je_Tour = {h: Umweg[h] / origin_dest_km[h] for h in Touren_sorted}
    ave_Umweg_je_Tour = np.mean(list(Umweg_je_Tour.values()))
    
    # Distance for unused drivers (direct origin-destination)
    origin_dest_km2 = {h: round(d[origin[h], destination[h]], 2) for h in H if h not in Touren_neu}
    
    # Total distance driven by all drivers (active + inactive)
    gefahrene_km = sum(ges_km_je_Tour[h] for h in ges_km_je_Tour) + sum(origin_dest_km2[h] for h in origin_dest_km2)
    
    # Environmental externality: total driven km vs. theoretical minimum
    Externalitaet = gefahrene_km / sum(origin_dest_km[h] for h in H)
    
    print("Tour je Fahrer:", Touren_sorted)
    
    return (Touren_neu, Touren_sorted, used_origins, TW_LB_driver, TW_UB_driver, used_destinations, 
            activitis, opt_S_driver, Touren, active_nodes, Starting_times, Starting_times_formatted, 
            active_t, used_ETP, used_WTP, Auftrag_Fahrer, Senders_surplus, Couriers_surplus, 
            Umwege_summe, Umweg, Umweg_ges_km, ges_km_je_Tour, origin_dest_km, Umweg_Tour_Verhältnis, 
            km_Auftrag, Umweg_je_Tour, ave_Umweg_je_Tour, origin_dest_km2, gefahrene_km, Externalitaet)


# ============================================================================
# VISUALIZATION FUNCTIONS
# ============================================================================

def graph_map():
    """
    Create geographical route map visualization using Berlin shapefiles.
    
    VISUALIZATION ELEMENTS:
    - Yellow squares (H): Driver origin points
    - Orange circles: Customer pickup locations  
    - Red circles: Customer delivery locations
    - Brown diamonds: Driver destination points
    - Colored arrows: Routes colored by driver
    - Gray boundaries: Berlin district borders
    - Green overlay: Environmental zone
    
    The map shows the optimized routing solution overlaid on Berlin geography,
    making it easy to understand spatial efficiency and route quality.
    
    Returns:
        tuple: (edge_colors, color_palette, num_edges, driver_colors, figure)
    """
    file_path = os.path.join(os.getcwd(), "data_Berlin", "berlin_ortsteile.shp")
    districts = gpd.read_file(file_path)
    
    G = nx.DiGraph()
    G.add_nodes_from(V)
    G.add_edges_from(active)
    pos = {i: (AllX[i], AllY[i]) for i in V}
    
    fig1 = plt.figure(figsize=(16, 10))
    graph_ax = fig1.add_subplot(1, 1, 1, projection=ccrs.PlateCarree())
    graph_ax = plt.axes([0, 0, 1, 1], projection=ccrs.PlateCarree())
    
    # Add geographical features
    graph_ax.add_feature(cfeature.LAKES, edgecolor='black', alpha=0.5)
    graph_ax.add_feature(cfeature.RIVERS, edgecolor='black', alpha=0.9)
    
    # Add district labels
    for i in district_names:
        graph_ax.text(district_coordy[i], district_coordx[i], district_names[i], 
                     fontsize=8, color='grey', ha='center', va='top')
    
    # Plot district boundaries
    districts.boundary.plot(ax=graph_ax, color='gray', linewidth=1, alpha=0.5)
    graph_ax.add_feature(cfeature.NaturalEarthFeature(
        category='cultural', name='admin_0_boundary_lines_land', 
        scale='10m', facecolor=None))
    graph_ax.set_extent([13.27, 13.51, 52.450, 52.56], crs=ccrs.PlateCarree())
    
    # Add environmental zone
    file_path2 = os.path.join(os.getcwd(), "data_Berlin", "Umweltzone_Berlin.shp")
    Umweltzone = gpd.read_file(file_path2)
    Umweltzone = Umweltzone.set_crs("EPSG:25833")
    target_crs = ccrs.PlateCarree()    
    Umweltzone = Umweltzone.to_crs(target_crs.proj4_init)
    
    for idx, polygon in Umweltzone.iterrows():
        graph_ax.add_geometries([polygon['geometry']], crs=ccrs.PlateCarree(), 
                               facecolor='green', edgecolor='black', alpha=0.1)
    
    # Draw nodes
    options = {"edgecolors": "k", "node_size": 200, "alpha": 0.5}
    nx.draw_networkx_nodes(G, pos, nodelist=tau[1:], node_color="yellow", node_shape='H', **options)
    nx.draw_networkx_nodes(G, pos, nodelist=A, node_color="tab:orange", node_shape='o', **options)
    nx.draw_networkx_nodes(G, pos, nodelist=A_, node_color="tab:red", node_shape='o', **options)
    nx.draw_networkx_nodes(G, pos, nodelist=tau_[1:], node_color="tab:brown", node_shape='d', **options)
    nx.draw_networkx_labels(G, pos, labels={n: n for n in G}, font_size=8, font_color='k')
    
    # Color edges by driver
    edges = list(G.edges())
    num_edges = len(edges) + 1
    colormap = plt.colormaps['tab20']
    colores = [colormap(i / num_edges) for i in range(max(Touren) + 1)]
    edgecolors = {a: colores[h] for h in Touren for a in Touren[h]}
    driver_color = {h: colores[h] for h in active_driver}
    edge_color_list = [edgecolors[a] for a in edges]
    
    # Draw edges
    nx.draw_networkx_edges(G, pos, edge_color=edge_color_list, alpha=1.0, width=1.5, 
                          arrows=True, arrowsize=12, ax=graph_ax, connectionstyle="arc3,rad=0.1")
    
    # Configure display
    graph_ax.set_xticks([])
    graph_ax.set_yticks([])
    graph_ax.set_xlim(13.27, 13.51)
    graph_ax.set_ylim(52.450, 52.562)
    graph_ax.gridlines(draw_labels=True)
    graph_ax.add_feature(cfeature.LAND)
    plt.title("Tourenmap:IPIC_SP_p")
    plt.show()
    
    return edgecolors, colores, num_edges, driver_color, fig1


def graph_gantt():
    """
    Create Gantt chart visualization of driver schedules and timing.
    
    GANTT CHART ELEMENTS:
    - Solid colored bars: Active service time at nodes
    - Light colored bars: Driver availability time windows
    - Hatched areas: Travel time between nodes
    - Y-axis: Driver IDs
    - X-axis: Time of day (10:00 - 20:00)
    
    This visualization helps understand:
    - When each driver is active vs. idle
    - How well time windows are utilized
    - Sequencing of pickup and delivery operations
    - Potential scheduling conflicts or inefficiencies
    
    Returns:
        tuple: (ending_times, formatted_ending_times, gantt_dataframe, service_times, figure)
    """    
    # Calculate ending times
    Ending_times = {}
    Ending_times = {i: (Starting_times[i] + s[i] + active_t[i]) 
                   for i in Starting_times if i not in tau_[1:]}
    Ending_times2 = {i: Starting_times[i] for i in tau_[1:] if i in active_nodes}
    Ending_times.update(Ending_times2)
    
    # Format ending times
    Ending_times_formatted = {}
    for key, value in Ending_times.items():
        time_obj = timedelta(minutes=value)
        time_str = (datetime.min + time_obj).strftime('%H:%M')
        Ending_times_formatted[key] = time_str
    
    # Calculate service times
    start_3 = {i: Starting_times[i] + s[i] for i in Starting_times if i not in tau_[1:]}
    start_32 = {i: Starting_times[i] for i in tau_[1:] if i in active_nodes}
    start_3.update(start_32)
    
    start3 = {i: 0 for i in Starting_times}
    for key, value in start_3.items():
        time_obj = timedelta(minutes=value)
        time_str = (datetime.min + time_obj).strftime('%H:%M')
        start3[key] = time_str
    
    end_3 = {i: Starting_times[i] + s[i] + active_t[i] for i in Starting_times if i not in tau_[1:]}
    end_32 = {i: Starting_times[i] for i in tau_[1:] if i in active_nodes}
    end_3.update(end_32)
    
    end3 = {i: 0 for i in Starting_times}
    for key, value in end_3.items():
        time_obj = timedelta(minutes=value)
        time_str = (datetime.min + time_obj).strftime('%H:%M')
        end3[key] = time_str
    
    # Create DataFrame for Gantt chart
    df = []
    for h, i in Touren_sorted.items():
        for j in i:
            df.append({
                'task': j, 
                'Start': Starting_times_formatted[j], 
                'Finish': Ending_times_formatted[j], 
                'Resource': h, 
                'color': driver_color[h], 
                'start2': TW_LB_driver[h], 
                'end2': TW_UB_driver[h], 
                'start3': start3[j], 
                'end3': end3[j]
            })
    
    # Create Gantt chart
    fig2, gantt_ax = plt.subplots(figsize=(16, 8))
    
    for item in df:
        start = datetime.strptime(item['Start'], '%H:%M')
        finish = datetime.strptime(item['Finish'], '%H:%M')
        duration = finish - start
        text_x2 = start + duration / 5
        text_y2 = item['Resource']
        
        gantt_ax.broken_barh([(start, duration)], (item['Resource'] - 0.25, 0.5), 
                            facecolors=item['color'], zorder=3, alpha=0.5)
        gantt_ax.annotate(item['task'], (text_x2, text_y2), xytext=(0, 0), 
                         textcoords='offset points', ha='center', va='bottom', 
                         fontsize=9, color='black', zorder=5)
        
        start2 = datetime.strptime(item['start2'], '%H:%M')
        end2 = datetime.strptime(item['end2'], '%H:%M')
        start3 = datetime.strptime(item['start3'], '%H:%M')
        end3 = datetime.strptime(item['end3'], '%H:%M')
        duration2 = end2 - start2
        duration3 = end3 - start3
        
        gantt_ax.broken_barh([(start2, duration2)], (item['Resource'] - 0.25, 0.5), 
                            facecolors=item['color'], zorder=2, alpha=0.1)
        gantt_ax.broken_barh([(start3, duration3)], (item['Resource'] - 0.25, 0.5), 
                            facecolor='none', edgecolor='lightgrey', hatch='//', zorder=4)
    
    # Create legend
    Tourdauer = mpatches.Patch(color='tab:red', label='Servicezeit am Knoten')
    Fahrer_ZF = mpatches.Patch(color='pink', label='Zeitfenster der Fahrer in hell')
    Fahrtzeit = mpatches.Patch(facecolor='none', hatch='//', edgecolor='grey', label='Fahrtzeit')
    Knoten_nummer = mpatches.Patch(color='white', label='1,2,.: Nr besuchter Knoten ')
    gantt_ax.legend(handles=[Tourdauer, Fahrer_ZF, Fahrtzeit, Knoten_nummer], loc='lower right')    
    
    plt.xlabel("Zeit")
    plt.ylabel("Fahrer")
    plt.title("Gantt:IPIC_SP_p")
    
    y_ticks = list(Touren_sorted.keys())
    y_labels = list(Touren_sorted.keys())  
    y_pos = list(Touren_sorted.keys())
    gantt_ax.set_yticks(y_pos, y_ticks)
    gantt_ax.set_yticklabels(y_labels) 
    
    x_ticks = pd.date_range(start=datetime(1900, 1, 1, 10, 0), 
                           end=datetime(1900, 1, 1, 20, 00), freq='h')
    x_labels = [time.strftime('%H:%M') for time in x_ticks]
    gantt_ax.set_xticks(x_ticks)
    gantt_ax.set_xticklabels(x_labels, rotation=45, ha='right')
    gantt_ax.grid(zorder=0)
    plt.show()
    
    return Ending_times, Ending_times_formatted, df, start3, fig2


# ============================================================================
# MAIN EXECUTION WORKFLOW
# ============================================================================

"""
EXECUTION FLOW EXPLANATION:

1. DATA LOADING: Read Berlin instance data (requests, drivers, time windows)
2. SET CREATION: Build mathematical sets and distance matrices  
3. OPTIMIZATION: Solve MIP model to find optimal routes and prices
4. RESULTS EXTRACTION: Process solution into business metrics
5. ADDITIONAL ANALYSIS: Calculate efficiency and surplus metrics
6. VISUALIZATION: Create map and Gantt chart of solution

The model optimizes both operational decisions (which routes to use) and 
financial decisions (what prices to charge), balancing platform profit
with customer satisfaction and driver compensation.
"""

# Load data and create optimization sets
print("Step 1: Loading Berlin instance data...")
all_data = read_data_create_sets(data1)
# Unpack all data components for model
print("Step 2: Creating optimization sets and parameters...")
(WTP_Faktor, ETP_Faktor, H, R, tau, tau_, V, pickup_delivery, ETP, WTP, ZF, TW_LB, TW_UB, 
 destination, origin, pickup_destination, delivery_origin, delivery_destination, Vh, 
 origin_pickup_delivery, pickup_delivery_destination, E, E1, d, d2, t, m, n, A, A_, 
 a, b, l, s, R2, AllX, AllY, Speed, district_names, district_coordx, district_coordy) = all_data

# Run the core optimization model
print("Step 3: Solving optimization model...")
print(f"Instance size: {n} requests, {m} drivers")
result = pricing_model(n, m, A, A_, H, WTP, ETP, R, d2, V, d, t, s, l, a, b, M)

if result is not None:
    # Unpack optimization results
    OF_p, optimal_L, optimal_S, optimal_c, optimal_p, optimal_x, optimal_z, Runtime1 = result
    print(f"✅ Optimization successful! Platform profit: €{OF_p:.2f}")
    
    # Extract and analyze business metrics
    print("Step 4: Extracting business metrics...")
    result2 = results()
    (active_arcs, active_prices, active_costs, active_load, active_z, active_Start, Touren, 
     active_driver, active_arcs2D, active, chosen_c, chosen_cost, chosen_p, opt_request_p, 
     opt_driver_c, Einnahmen, Ausgaben, used_driver, possible_matches, matches, 
     Kantenkosten, Marge_je_Auftrag, Marge_je_Tour, aktive_Auftraege) = result2

    # Process additional efficiency metrics
    print("Step 5: Calculating efficiency and surplus metrics...")
    result3 = variables_part2()
    (Touren_neu, Touren_sorted, used_origins, TW_LB_driver, TW_UB_driver, used_destinations, 
     activitis, opt_S_driver, Touren, active_nodes, Starting_times, Starting_times_formatted, 
     active_t, used_ETP, used_WTP, Auftrag_Fahrer, Senders_surplus, Couriers_surplus, 
     Umwege_summe, Umweg, Umweg_ges_km, ges_km_je_Tour, origin_dest_km, Umweg_Tour_Verhältnis, 
     km_Auftrag, Umweg_je_Tour, ave_Umweg_je_Tour, origin_dest_km2, gefahrene_km, 
     Externalitaet) = result3

    # Create geographical visualization
    print("Step 6: Creating route map visualization...")
    result4 = graph_map()
    edgecolors, colores, num_edges, driver_color, fig1 = result4

    # Create temporal visualization  
    print("Step 7: Creating Gantt chart visualization...")
    result5 = graph_gantt()
    Ending_times, Ending_times_formatted, df, start3, fig2 = result5
    
    print("🎯 Analysis complete! Check the visualizations above.")
    
    # ========================================================================
    # FINAL SUMMARY REPORT
    # ========================================================================
    
    print("\n" + "=" * 80)
    print("CROWDSHIPPING PLATFORM PERFORMANCE SUMMARY")
    print("=" * 80)
    print(f"📊 OPERATIONAL EFFICIENCY:")
    print(f"   • Match Rate: {matches}/{possible_matches} requests ({matches/possible_matches*100:.1f}%)")
    print(f"   • Driver Utilization: {len(active_driver)}/{len(H)} drivers ({len(active_driver)/len(H)*100:.1f}%)")
    print(f"   • Average Detour Ratio: {ave_Umweg_je_Tour:.2f}x direct distance")
    print(f"   • Environmental Impact: {Externalitaet:.2f}x optimal distance")
    print()
    print(f"💰 FINANCIAL PERFORMANCE:")
    print(f"   • Platform Profit: €{OF_p:.2f}")
    print(f"   • Total Revenue: €{Einnahmen:.2f}")
    print(f"   • Total Costs: €{Ausgaben:.2f}")
    print(f"   • Customer Surplus: €{Senders_surplus:.2f}")
    print(f"   • Driver Surplus: €{Couriers_surplus:.2f}")
    print()
    print(f"🚗 ROUTING EFFICIENCY:")
    print(f"   • Total Distance Driven: {gefahrene_km:.2f} km")
    print(f"   • Total Detours: {Umwege_summe:.2f} km")
    print(f"   • Average Requests per Active Driver: {matches/len(active_driver):.1f}" if active_driver else "   • No active drivers")
    print("=" * 80)

else:
    print("❌ Optimization failed - model may be infeasible with current parameters.")
    print("💡 Troubleshooting suggestions:")
    print("   • Check if time windows allow feasible solutions")
    print("   • Verify that WTP values are reasonable vs. distances") 
    print("   • Ensure driver capacities are sufficient for request loads")
    print("   • Consider relaxing ETP constraints if too restrictive")